In [1]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier as knn

#LOading the datasets

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
bstruct=pd.read_csv('Building_Structure.csv')
bown=pd.read_csv('Building_Ownership_Use.csv')

In [ ]:
#Mapping the object datatype to int

In [6]:
final_data=[train,test]
train=pd.merge(train,bstruct)
zz=['land_surface_condition','foundation_type','roof_type','ground_floor_type','other_floor_type','position','plan_configuration','condition_post_eq']

test=pd.merge(test,bstruct)
dictionary=[]
for col in zz:
    key=bstruct[[col]].groupby([col]).groups.keys()
    dctio={}
    for c,d in enumerate(key):
        dctio[d]=c
    dictionary.append(dctio)
    train[col]=train[col].map(dctio)
    test[col]=test[col].map(dctio)
zz=['legal_ownership_status']
train=pd.merge(train,bown)
test=pd.merge(test,bown)
dictionary=[]
for col in zz:
    key=bown[[col]].groupby([col]).groups.keys()
    dctio={}
    for c,d in enumerate(key):
        dctio[d]=c
    dictionary.append(dctio)
    train[col]=train[col].map(dctio)
    test[col]=test[col].map(dctio)


In [7]:
train['damage_grade']=train.damage_grade.str.extract('([\d])')
#test['damage_grade']=test.damage_grade.str.extract('([\d])')
build=test['building_id']

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [8]:
mapp={'Interior':0,'Exterior':1,'Both':2,'Building removed':3,'Not able to inspect':4}

train['area_assesed']=train['area_assesed'].map(mapp)
test['area_assesed']=test['area_assesed'].map(mapp)
df=train.groupby(['area_assesed'])
print(df.head())


      area_assesed   building_id damage_grade  district_id  \
0                2   24385bfd2a2            4           24   
1                2   405d1bbebbf            2           44   
2                2   351d9bc71f6            1           36   
3                3   2be3a971166            5           30   
4                2   34c7d073ea6            3           36   
5                1   1f54965821f            5           21   
6                2   2beb42eaeee            2           30   
10               1   35074b0c35f            5           36   
13               3   355539e8f10            5           36   
19               1  1b9136a5a0af            3           30   
23               3   4372e823ef3            5           46   
26               3   1d7b8343e6a            5           20   
27               3   1f45e8dec7e            5           21   
30               1   2c3594bef82            4           30   
31               1   2d635dfaa18            2           31   
98      

In [9]:
trainy=train['damage_grade']
trainx=train.drop(['building_id','damage_grade'],1)
test=test.drop(['building_id'],1)

In [ ]:
#Fixing the NaN values

In [10]:
from sklearn.preprocessing import Imputer
imp=Imputer()
trainx['count_families']=imp.fit_transform(trainx[['count_families']])
test['count_families']=imp.fit_transform(test[['count_families']])
trainx['has_repair_started']=imp.fit_transform(trainx[['has_repair_started']])
test['has_repair_started']=imp.fit_transform(test[['has_repair_started']])

In [11]:
print(test.isnull().any())

area_assesed                              False
district_id                               False
has_geotechnical_risk                     False
has_geotechnical_risk_fault_crack         False
has_geotechnical_risk_flood               False
has_geotechnical_risk_land_settlement     False
has_geotechnical_risk_landslide           False
has_geotechnical_risk_liquefaction        False
has_geotechnical_risk_other               False
has_geotechnical_risk_rock_fall           False
has_repair_started                        False
vdcmun_id                                 False
ward_id                                   False
count_floors_pre_eq                       False
count_floors_post_eq                      False
age_building                              False
plinth_area_sq_ft                         False
height_ft_pre_eq                          False
height_ft_post_eq                         False
land_surface_condition                    False
foundation_type                         

In [32]:
drop=['has_secondary_use_agriculture','has_secondary_use_hotel','has_secondary_use_rental','has_secondary_use_institution','has_secondary_use_school','has_secondary_use_industry','has_secondary_use_health_post','has_secondary_use_gov_office','has_secondary_use_use_police','has_secondary_use_other']
trainx=trainx.drop(drop,1)
test=test.drop(drop,1)

In [37]:
# drop=['legal_ownership_status','count_families']
# trainx=trainx.drop(drop,1)
# test=test.drop(drop,1)
from sklearn.ensemble import RandomForestClassifier as rc
from sklearn.model_selection import train_test_split as tts
trax,tesx,tray,tesy=tts(trainx,trainy,test_size=0.2)

In [13]:
# new=pd.merge(train,bown)
# print(new.head(5),new.describe(),train.describe())

In [14]:
# print(bown.dtypes)

In [15]:
# zz=['legal_ownership_status']
# #print(bstruct[zz].head(5))
# lsc={'Moderate slope':0,'Steep slope':1,'Flat':2}
# fdt={'Mud mortar-Stone/Brick':0,'Cement-Stone/Brick':1,'Bamboo/Timber':2,'RC':3,'Other':4}
# roof={'Bamboo/Timber-Light roof':0,'RCC/RB/RBC':1,'Bamboo/Timber-Heavy roof':2}
# a=bstruct[['ground_floor_type']].groupby(['ground_floor_type']).groups.keys()
# #for c,d in enumerate(a):
# #    print(c,d)
# #print(roof)
# dictionary=[]
# for col in zz:
#     key=bown[[col]].groupby([col]).groups.keys()
#     dctio={}
#     for c,d in enumerate(key):
#         dctio[d]=c
#     dictionary.append(dctio)
# print(dictionary)

In [24]:
from sklearn.linear_model import LinearRegression as ln

In [36]:
trax.shape

(505408, 49)

In [39]:
model=rc()
model.fit(trainx,trainy)
pred=model.predict(test)
score=model.score(tesx,tesy)
print(score)

0.9876536370327574


In [40]:
for i in range(0,len(pred)-1):
    pred[i]='Grade '+str(pred[i])
# print(score)
my_submission = pd.DataFrame({'building_id': build, 'damage_grade': pred})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

In [18]:
# a=[2,'df4',5,6]
# a[0]='grade'+str(a[0])
# print(a)